In [2]:
import requests
import pandas as pd
import ast
from rapidfuzz import process, fuzz
import json
import csv
import numpy as np

platforms_df
images1

In [2]:
consoles_list=['Game Boy','Super Nintendo Entertainment System','Nintendo 64','PlayStation 5','Nintendo Switch 2','Xbox','PlayStation 2','Game Boy Color',
              'Nintendo Switch','Xbox Series X|S','Next Generation Xbox','Game Boy Advance','GameCube','Nintendo DS','Xbox 360','PlayStation 3','Wii','Nintendo 3DS', 
              'PlayStation 4'
             ]

In [13]:
g1=pd.read_csv('../data/games1_1.csv')

In [14]:
g2=pd.read_csv('../data/games2_1.csv')

In [15]:
g10=pd.read_csv('../data/games2_2.csv')

In [16]:
g11=pd.read_csv('../data/games3.csv')

In [17]:
g12=pd.read_csv('../data/games4.csv')

In [18]:
g13=pd.read_csv('../data/games2_3.csv')

In [19]:
g3=pd.concat([g1,g2,g10,g11,g12,g13],axis=0,ignore_index=True)
g3

,Unnamed: 0,deck,id,image,name,number_of_user_reviews,original_release_date,platforms,characters,concepts,...,first_appearance_characters,first_appearance_concepts,first_appearance_locations,first_appearance_objects,franchises,genres,killed_characters,locations,publishers,themes
0,0,A top-down isometric helicopter shoot 'em up o...,1,{'icon_url': 'https://www.giantbomb.com/a/uplo...,Desert Strike: Return to the Gulf,0,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
1,1,A text adventure game created by Sam Barlow. O...,19,{'icon_url': 'https://www.giantbomb.com/a/uplo...,Aisle,0,1999-05-28,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,NaN
2,2,"This game, based on Star Wars Episode IV: A Ne...",54,{'icon_url': 'https://www.giantbomb.com/a/uplo...,Star Wars,0,1991-11-15,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
3,3,Choplifter II was the first Choplifter title f...,111,{'icon_url': 'https://www.giantbomb.com/a/uplo...,Choplifter II: Rescue Survive,0,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,NaN,...,NaN,NaN,NaN,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN
4,4,NaN,120,{'icon_url': 'https://www.giantbomb.com/a/uplo...,Nail'N Scale,0,1992-12-31,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,NaN,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9094,995,"From the creators of Abzû and The Pathless, Sw...",89238,{'icon_url': 'https://www.giantbomb.com/a/uplo...,Sword of the Sea,0,2025-08-19,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,NaN,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,[{'api_detail_url': 'https://www.giantbomb.com...
9095,996,Foamstars is a 4-on-4 arena shooter developed ...,89241,{'icon_url': 'https://www.giantbomb.com/a/uplo...,Foamstars,0,2024-02-06,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,NaN,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN
9096,997,A full rem

In [6]:
game_reviews=pd.read_csv('../data/reviews_and_sales.csv')

In [7]:
g3['name'] = g3['name'].astype(str)
game_reviews['Name'] = game_reviews['Name'].astype(str)

# Compute similarity matrix in parallel
scores = process.cdist(
    game_reviews['Name'].tolist(),
    g3['name'].tolist(),
    scorer=fuzz.token_sort_ratio,
    workers=-1  # use all CPU cores
)

# Get best matches and their scores
best_match_idx = np.argmax(scores, axis=1)
best_match_score = scores[np.arange(scores.shape[0]), best_match_idx]
best_matches = [g3['name'].iloc[i] for i in best_match_idx]

# Add results back to reviews_and_sales
game_reviews['matched_name'] = best_matches
game_reviews['score'] = best_match_score

# Replace Name with matched_name if score >= threshold
threshold = 85
game_reviews.loc[game_reviews['score'] >= threshold, 'Name'] = \
   game_reviews.loc[game_reviews['score'] >= threshold, 'matched_name']

# Merge on aligned names
merged_df = pd.merge(
    g3,
    game_reviews,
    left_on='name',
    right_on='Name',
    how='inner'
)

# merged_df now contains both datasets combined
merged_df.head(1)


,Unnamed: 0,deck,id,image,name,number_of_user_reviews,original_release_date,platforms,characters,concepts,...,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,matched_name,score
0,5,A spin-off of the NBA Jam arcade basketball se...,125,{'icon_url': 'https://www.giantbomb.com/a/uplo...,College Slam,0,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,NaN,[{'api_detail_url': 'https://www.giantbomb.com...,...,0.01,0.19,NaN,NaN,NaN,NaN,NaN,NaN,College Slam,100.0


In [8]:
g4=merged_df

In [9]:
g4["platforms"] = g4["platforms"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [10]:
# Step 1: extract just the names of platforms per row
g4["platform_names"] = g4["platforms"].apply(
    lambda lst: [d.get("name") for d in lst] if isinstance(lst, list) else []
)

# Step 2: build indicator columns for each console in your list
console_games_df = pd.DataFrame(index=g4.index)

for console in consoles_list:
    console_games_df[console] = g4["platform_names"].apply(
        lambda names: console if console in names else "-"
    )

# Done: console_games_df now has one row per game, columns = consoles
console_games_df

,Game Boy,Super Nintendo Entertainment System,Nintendo 64,PlayStation 5,Nintendo Switch 2,Xbox,PlayStation 2,Game Boy Color,Nintendo Switch,Xbox Series X|S,Next Generation Xbox,Game Boy Advance,GameCube,Nintendo DS,Xbox 360,PlayStation 3,Wii,Nintendo 3DS,PlayStation 4
0,Game Boy,Super Nintendo Entertainment System,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,Game Boy,Super Nintendo Entertainment System,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,Game Boy,Super Nintendo Entertainment System,-,-,-,-,PlayStation 2,-,Nintendo Switch,-,-,Game Boy Advance,-,Nintendo DS,-,-,-,-,-
3,Game Boy,Super Nintendo Entertainment System,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,Game Boy,Super Nintendo Entertainment System,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Xbox 360,-,-,-,PlayStation 4
1777,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Xbox 360,-,-,-,PlayStation 4
1778,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Xbox 360,-,-,-,PlayStation 4
1779,-,-,-,-,-,-,-,-,-,-,-,-,-,-,Xbox 360,-,-,-,PlayStation 4


In [11]:
g5=pd.concat([g4,console_games_df],axis=1)

In [12]:
cols = ['characters', 'image','concepts','developers','first_appearance_characters','first_appearance_concepts',
        'first_appearance_locations','first_appearance_objects','franchises','genres','locations','publishers','themes']

for col in cols:
    def extract_names(x):
        # Convert string to list if needed
        if isinstance(x, str):
            try:
                x = ast.literal_eval(x)
            except:
                return ''
        # If it's a list of dicts, extract 'name'
        if isinstance(x, list):
            names = [d.get('name','') for d in x if isinstance(d, dict)]
            return ', '.join(names)
        return ''
    
    g5[col + '_name'] = g5[col].apply(extract_names)

In [17]:
g5.to_csv('first_games.csv')

In [16]:
g5['image_name']

0        
1        
2        
3        
4        
       ..
1776     
1777     
1778     
1779     
1780     
Name: image_name, Length: 1781, dtype: object

In [ ]:
g5.loc[g5[['original_release_date','Year_of_Release','NA_Sales','EU_Sales','JP_Sales','Other_Sales','Global_Sales','Critic_Score','User_Count','Score']]!=int]

In [27]:
cols = ['Year_of_Release','NA_Sales','EU_Sales','JP_Sales',
        'Other_Sales','Global_Sales','Critic_Score','User_Count','score']

# Function to check if a value is numeric
def is_number(x):
    try:
        float(x)
        return True
    except (ValueError, TypeError):
        return False

# Create a mask where any column is non-numeric
mask = ~g5[cols].applymap(is_number).all(axis=1)

# Select the problematic rows
error_rows = g5[mask]

print(error_rows)

Empty DataFrame
Columns: [Unnamed: 0, deck, id, image, name, number_of_user_reviews, original_release_date, platforms, characters, concepts, developers, first_appearance_characters, first_appearance_concepts, first_appearance_locations, first_appearance_objects, franchises, genres, killed_characters, locations, publishers, themes, Name, Platform, Year_of_Release, Genre, Publisher, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales, Critic_Score, Critic_Count, User_Score, User_Count, Developer, Rating, matched_name, score, platform_names, Game Boy, Super Nintendo Entertainment System, Nintendo 64, PlayStation 5, Nintendo Switch 2, Xbox, PlayStation 2, Game Boy Color, Nintendo Switch, Xbox Series X|S, Next Generation Xbox, Game Boy Advance, GameCube, Nintendo DS, Xbox 360, PlayStation 3, Wii, Nintendo 3DS, PlayStation 4, characters_name, image_name, concepts_name, developers_name, first_appearance_characters_name, first_appearance_concepts_name, first_appearance_locations_name, firs

C:\Users\asher\AppData\Local\Temp\ipykernel_79920\3169060507.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = ~g5[cols].applymap(is_number).all(axis=1)


In [28]:
error_rows[cols]

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,User_Count,score


In [25]:
len(g5)

1781